这次能跑起来就算成功，但是最后得到的远远超出我的预期，所以在此逐步总结，好彻底结尾

首先在这里感谢gpt爹的全力帮助，感谢你不厌其烦地给我解答，谢谢你


## 数据集准备

我是在kaggle上面搜索的数据集，是ted的演讲csv，搜索数据集应该不算难事，我仅仅在这里附上网站
https://www.kaggle.com/datasets/rounakbanik/ted-talks

### 数据清洗

我想下面一个函数能解释一切

In [ ]:
def clean_text(text):
    # 去除所有 (...) 和 [...] 中的内容，包括括号本身
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'\[[^\]]*\]', '', text)
    # 替换多个空格为一个空格
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

我把csv放到一个文件夹里面了方便集中处理

In [ ]:
# 遍历文件夹下所有 CSV 文件
for csv_file in csv_dir.glob("*.csv"):
    df = pd.read_csv(csv_file)
    # 自动选择合适的文本列
    column = 'transcript' if 'transcript' in df.columns else 'content' if 'content' in df.columns else None
    if column is None:
        print(f"⚠️ 跳过文件（无合适文本列）: {csv_file.name}")
        continue


具体实现在代码里面有，再次感谢gpt爹

### 数据预处理

这里已经得到了初步可用的数据，但是为了增强数据可用性，需要进行预处理增强

In [ ]:
# 2. 添加特殊 token：<s> 开头，<eos> 结尾
# 将每一行处理为句子（可根据需要替换为其他分割方法）
lines = raw_text.strip().splitlines()
tokenized_lines = [["<s>"] + line.strip().split() + ["<eos>"] for line in lines]
words = [word for line in tokenized_lines for word in line]

In [ ]:
# 4. 保存词表
with open("transformer!/gpt/vocab.pkl", "wb") as f:
    pickle.dump((stoi, itos), f)

用的词嵌入的方法，得到了词表vocab.pkl，这个文件在transformer/gpt文件夹下,但是还不够，需要将其使其编码

In [ ]:
# 5. 编码文本为 token ID
def encode(word_list):
    return [stoi.get(w, stoi["<unk>"]) for w in word_list]

data_ids = torch.tensor(encode(words), dtype=torch.long)

然后才是划分训练集和验证集得到train和val

In [ ]:
# 6. 划分训练/验证集
n = int(0.9 * len(data_ids))
train_data = data_ids[:n]
val_data = data_ids[n:]

# 7. 保存编码后的数据
Path("transformer!/gpt/data").mkdir(parents=True, exist_ok=True)
torch.save(train_data, "transformer!/gpt/data/train.pt")
torch.save(val_data, "transformer!/gpt/data/val.pt")

## 模型的构建

定义好参数的东西

In [ ]:
class GPTConfig:
    # 初始化方法，用于创建GPTConfig类的实例
    def __init__(self, vocab_size, block_size=128, n_layer=4, n_head=4, n_embd=256, dropout=0.1):
        # vocab_size: 词汇表大小，表示模型可以处理的唯一标记的数量
        self.vocab_size = vocab_size
        # block_size: 上下文窗口大小，表示模型在一次前向传播中可以处理的序列长度
        self.block_size = block_size
        # n_layer: Transformer层的数量，表示模型的深度
        self.n_layer = n_layer
        # n_head: 多头注意力机制中的注意力头数，表示每个注意力头负责的部分注意力计算
        self.n_head = n_head
        # n_embd: 嵌入维度，表示每个标记的向量表示的维度
        self.n_embd = n_embd
        # dropout: Dropout率，用于防止过拟合，表示在训练过程中随机丢弃的比例
        self.dropout = dropout

关键的自注意力机制

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, config):
        # 初始化父类
        super().__init__()
        # 定义线性变换层，用于计算key
        self.key = nn.Linear(config.n_embd, config.n_embd)
        # 定义线性变换层，用于计算query
        self.query = nn.Linear(config.n_embd, config.n_embd)
        # 定义线性变换层，用于计算value
        self.value = nn.Linear(config.n_embd, config.n_embd)
        # 定义线性变换层，用于输出投影
        self.proj = nn.Linear(config.n_embd, config.n_embd)
        # 定义dropout层，用于防止过拟合
        self.dropout = nn.Dropout(config.dropout)
        # 定义注意力头的数量
        self.n_head = config.n_head

    def forward(self, x):
        # 获取输入的批次大小、序列长度和嵌入维度
        B, T, C = x.size()
        # 计算每个注意力头的维度
        head_dim = C // self.n_head
        # 计算key，并调整形状以适应多头注意力
        k = self.key(x).view(B, T, self.n_head, head_dim).transpose(1, 2)
        # 计算query，并调整形状以适应多头注意力
        q = self.query(x).view(B, T, self.n_head, head_dim).transpose(1, 2)
        # 计算value，并调整形状以适应多头注意力
        v = self.value(x).view(B, T, self.n_head, head_dim).transpose(1, 2)

        # 计算注意力分数
        att = (q @ k.transpose(-2, -1)) / (head_dim ** 0.5)
        # 创建掩码矩阵，用于防止未来信息的泄露
        mask = torch.tril(torch.ones(T, T, device=x.device)).unsqueeze(0).unsqueeze(0)
        # 将注意力分数中不需要的部分设为负无穷
        att = att.masked_fill(mask == 0, float('-inf'))
        # 应用softmax函数，得到注意力权重
        att = F.softmax(att, dim=-1)
        # 应用dropout
        att = self.dropout(att)
        # 计算输出
        out = att @ v
        # 调整输出形状
        out = out.transpose(1, 2).contiguous().view(B, T, C)
        # 应用输出投影
        return self.proj(out)

然后一个完整的transformer块

In [ ]:
class TransformerBlock(nn.Module):
    # 定义TransformerBlock类，继承自nn.Module
    def __init__(self, config):
        # 初始化函数，接收配置参数config
        super().__init__()
        # 调用父类nn.Module的初始化函数
        self.sa = SelfAttention(config)
        # 初始化自注意力机制模块SelfAttention，传入配置参数config
        self.ln1 = nn.LayerNorm(config.n_embd)
        # 初始化第一个层归一化模块LayerNorm，输入维度为config.n_embd
        self.ff = nn.Sequential(
            # 初始化前馈神经网络模块，包含以下层：
            nn.Linear(config.n_embd, 4 * config.n_embd),
            # 线性层，将输入维度从config.n_embd变换到4倍的config.n_embd
            nn.ReLU(),
            # ReLU激活函数
            nn.Linear(4 * config.n_embd, config.n_embd),
            # 线性层，将输入维度从4倍的config.n_embd变换回config.n_embd
            nn.Dropout(config.dropout),
            # Dropout层，防止过拟合，丢弃率为config.dropout
        )
        self.ln2 = nn.LayerNorm(config.n_embd)

        # 初始化第二个层归一化模块LayerNorm，输入维度为config.n_embd
    def forward(self, x):
        # 定义前向传播函数，接收输入x
        x = x + self.sa(self.ln1(x))
        # 计算自注意力机制输出，并进行残差连接，输入x经过第一个层归一化后传入自注意力模块
        x = x + self.ff(self.ln2(x))
        # 计算前馈神经网络输出，并进行残差连接，输入x经过第二个层归一化后传入前馈神经网络模块
        return x

完整的类

In [ ]:
class GPT(nn.Module):
    def __init__(self, config):
        # 初始化父类nn.Module
        super().__init__()
        # 创建词嵌入层，将词汇表中的每个词映射到n_embd维的向量
        self.token_embed = nn.Embedding(config.vocab_size, config.n_embd)
        # 创建位置嵌入层，将序列中的每个位置映射到n_embd维的向量
        self.pos_embed = nn.Embedding(config.block_size, config.n_embd)
        # 创建多个TransformerBlock，并使用nn.Sequential将它们串联起来
        self.blocks = nn.Sequential(*[TransformerBlock(config) for _ in range(config.n_layer)])
        # 创建层归一化层，用于对最后的输出进行归一化处理
        self.ln_f = nn.LayerNorm(config.n_embd)
        # 创建线性层，将n_embd维的向量映射到词汇表大小的向量，用于生成预测
        self.head = nn.Linear(config.n_embd, config.vocab_size)

    def forward(self, idx):
        # 获取输入的批次大小B和序列长度T
        B, T = idx.size()
        # 创建一个从0到T-1的序列，表示位置信息，并将其移动到与输入相同的设备上
        pos = torch.arange(T, device=idx.device)
        # 将输入的索引通过词嵌入层和位置嵌入层，得到嵌入向量，并进行相加
        x = self.token_embed(idx) + self.pos_embed(pos)
        # 将嵌入向量通过多个TransformerBlock进行前向传播
        x = self.blocks(x)
        # 对最后的输出进行层归一化处理
        x = self.ln_f(x)
        # 将归一化后的输出通过线性层，得到最终的预测结果
        return self.head(x)

构建模型很简单啊是不是，其实都是gpt爹做的

## 准备训练

一些简单的配置，这里我用了cuda，没这个真的不行，还用了训练可视化tensorboard

In [ ]:
# 1. 环境配置
device = 'cuda' if torch.cuda.is_available() else 'cpu'
writer = SummaryWriter(log_dir="transformer!/gpt/runs")

In [ ]:
# 2. 加载词表
with open("transformer!/gpt/vocab.pkl", "rb") as f:
    stoi, itos = pickle.load(f)
vocab_size = len(stoi)

In [ ]:
# 3. 超参数配置
block_size = 64  # 句子长度（单词数）
n_layer = 8
n_head = 8
n_embd = 256
dropout = 0.1
lr = 3e-4
batch_size = 32
max_steps = 50000

超参数是最关键的部分，要根据要求酌情调整，我最后因为断电影响，只能训练五万个周期，gpu也支撑不了更高的参数，但是已经足够了

In [ ]:
# 4. 加载预处理数据
train_data = torch.load("transformer!/gpt/data/train.pt")
val_data = torch.load("transformer!/gpt/data/val.pt")


In [ ]:
# 5. 构建模型
config = GPTConfig(vocab_size, block_size, n_layer, n_head, n_embd, dropout)
model = GPT(config).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scaler = GradScaler()


In [ ]:
# 6. 生成训练批次
def get_batch(data):
    ix = torch.randint(0, len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)


In [ ]:
# 7. 训练循环
for step in range(max_steps):
    x, y = get_batch(train_data)

    # AMP 混合精度训练
    with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
        logits = model(x)
        loss = F.cross_entropy(logits.view(-1, vocab_size), y.view(-1))

    optimizer.zero_grad()
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    writer.add_scalar("Loss/Train", loss.item(), step)

    # 验证与打印
    if step % 100 == 0:
        model.eval()
        with torch.no_grad(), autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
            x_val, y_val = get_batch(val_data)
            val_logits = model(x_val)
            val_loss = F.cross_entropy(val_logits.view(-1, vocab_size), y_val.view(-1))
        writer.add_scalar("Loss/Val", val_loss.item(), step)
        print(f"Step {step}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")
        torch.cuda.empty_cache()
        model.train()

# 8. 保存模型
torch.save(model.state_dict(), "transformer!/gpt/gpt_model.pt")
print("✅ 训练完成，模型已保存")
writer.close()

## 然后是解读

In [ ]:
# 加载词表
with open("transformer!/gpt/vocab.pkl", "rb") as f:
    stoi, itos = pickle.load(f)

def encode(s):
    return [stoi.get(w, stoi.get('<unk>', 0)) for w in s.split()]

def decode(l):
    words = [itos[i] for i in l]
    return ' '.join(w for w in words if w not in {'<unk>', '<eos>', '<s>'})

这里有个刚才没讲到的关键东西unk, eos, s，有两个是用来断句的，还有一个用来处理陌生词汇，增强了鲁棒性

In [ ]:
def load_model(model_path, device, block_size=64):
    vocab_size = len(stoi)
    config = GPTConfig(
        vocab_size=vocab_size,
        block_size=block_size,
        n_layer=8,
        n_head=8,
        n_embd=256,
        dropout=0.1
    )
    model = GPT(config).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model, config

以及最最关键的一个函数

In [ ]:
@torch.no_grad()
def generate(model, config, prompt, device, max_new_tokens=80, temperature=0.7, top_p=0.85, do_sample=True):
    # 将输入的prompt编码为tensor，并移动到指定设备上
    input_ids = torch.tensor(encode(prompt), dtype=torch.long)[None, :].to(device)
    # 克隆input_ids以避免修改原始输入
    idx = input_ids.clone()
    # 获取输入的长度
    input_len = input_ids.shape[1]

    # 循环生成新的token，直到达到最大长度或遇到结束符
    for _ in range(max_new_tokens):
        # 获取当前输入的最后block_size个token
        idx_cond = idx[:, -config.block_size:]
        # 使用模型生成logits
        logits = model(idx_cond)
        # 对logits进行缩放
        logits = logits[:, -1, :] / temperature
        # 计算softmax概率
        probs = F.softmax(logits, dim=-1)

        # 如果选择采样生成
        if do_sample:
            # 对概率进行降序排序
            sorted_probs, sorted_indices = torch.sort(probs, descending=True)
            # 计算累积概率
            cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
            # 获取累积概率超过top_p的位置
            cutoff = cumulative_probs > top_p
            # 确保至少有一个token的概率不为0
            cutoff[..., 1:] = cutoff[..., :-1].clone()
            cutoff[..., 0] = False
            # 将超过top_p的token概率设为0
            sorted_probs[cutoff] = 0
            # 归一化概率
            sorted_probs = sorted_probs / sorted_probs.sum(dim=-1, keepdim=True)
            # 使用多项式分布采样下一个token
            next_token = torch.multinomial(sorted_probs, num_samples=1)
            # 获取对应的token索引
            next_token = sorted_indices.gather(-1, next_token)
        else:
            # 如果不采样，选择概率最大的token
            next_token = torch.argmax(probs, dim=-1, keepdim=True)

        # 将生成的token添加到当前序列中
        idx = torch.cat((idx, next_token), dim=1)

        # 如果生成 <eos>，提前结束
        if next_token.item() == stoi.get("<eos>", -1):
            break

    # 只返回生成的部分（不包括 prompt）
    generated = idx[0, input_len:]
    return decode(generated.tolist())

注意里面的参数，我没有特地拿出来，但实际上也是很重要的
max_new_tokens=80, temperature=0.7, top_p=0.85, do_sample=True

什么降温升温的，参数不懂可以问gpt，可以极大地影响生成的东西

最后的主函数

In [ ]:
if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model, config = load_model("transformer!/gpt/gpt_model.pt", device)

    while True:
        user_input = input("你：").strip()
        if user_input.lower() == "exit":
            break

        prompt = f"User: {user_input} Bot:"
        response = generate(model, config, prompt, device)
        print("🤖：" + response)

最后我除了gpt爹我还要感谢我自己，感谢牢卢，我从两年前一无所知，到一年前刚接触ai，再到现在终于能跑一个智障，真是太艰难了，所以我要感谢。
还有我的家人们，因为我的任性，买了很贵的电脑，这确实是一笔巨款，对我们家，但是他们还是买了，现在我终于用它做出了点成就，将来一定也能做更多，来回报一直以来的支持，感谢，爱你们